In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [2]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN, self).__init__()
    # N_out = floor( (N_in+2p-k)/2 ) + 1,
    # Where, N_in = number of input features
    #        N_out = number of output features
    #        k = convolution kernel size
    #        s = convolution stride size

    self.conv1 = nn.Conv2d(
        in_channels=in_channels,
        out_channels=8,
        kernel_size=(3, 3), # same convolution
        stride=(1, 1),
        padding=(1, 1),
    ) # 28 features
    self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # 14 features
    self.conv2 = nn.Conv2d(
        in_channels=8,
        out_channels=16,
        kernel_size=(3, 3), # same convolution
        stride=(1, 1),
        padding=(1, 1),
    )
    self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # 7 features
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x

In [3]:
model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)

torch.Size([64, 10])


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [6]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)

In [7]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: dataset/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [8]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
train_loader

In [9]:
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: dataset/
    Split: Test
    StandardTransform
Transform: ToTensor()

In [10]:
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
test_loader

In [11]:
model = CNN().to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [13]:
for epoch in tqdm(range(num_epochs)):
  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get the data to cuda if possible
    data = data.to(device=device) # 64, 1, 28, 28
    targets = targets.to(device=device)

    # Forward pass
    scores = model(data)

    # Calculate the loss
    loss = criterion(scores, targets)

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()


100%|██████████| 10/10 [02:17<00:00, 13.72s/it]


In [14]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(dim=1)

      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f"Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100}")

  model.train()

In [15]:
check_accuracy(train_loader, model)

Checking accuracy on training data
Got 59438/60000 with accuracy 99.06333333333333


In [16]:
check_accuracy(test_loader, model)

Checking accuracy on test data
Got 9853/10000 with accuracy 98.53
